# Import Packages

In [17]:
import json
import pandas as pd
import numpy as np

# Functions

In [18]:
def sub_df(df, column_values, column_name):
    #creates subset of dataframe consisting of rows with column values in column
    mask = df[column_name].apply(lambda x: any(value for value in column_values if value in x))
    return df[mask]

def load_main_monsters():
    with open('cardinfo.php') as file_path:
        cards_json = json.load(file_path)
    all_cards = pd.DataFrame(cards_json['data'])
    all_cards = all_cards.rename(columns={'type': 'card_type','race':'type'})

    main_monster_card_types = ['Effect Monster',
                                'Normal Monster',
                                'Flip Effect Monster',
                                'Union Effect Monster',
                                'Pendulum Effect Monster',
                                'Tuner Monster',
                                'Gemini Monster',
                                'Normal Tuner Monster',
                                'Spirit Monster',
                                'Ritual Effect Monster',
                                'Ritual Monster',
                                'Toon Monster',
                                'Pendulum Normal Monster',
                                'Pendulum Tuner Effect Monster',
                                'Pendulum Effect Ritual Monster',
                                'Pendulum Flip Effect Monster']
    MAIN_MONSTERS = sub_df(all_cards, main_monster_card_types, 'card_type').reset_index(drop=True)
    MAIN_MONSTERS = MAIN_MONSTERS[['name','type','attribute','level','atk','def']] #keep only relevant columns
    return MAIN_MONSTERS

def create_sw_adjacency_matrix(deck_mosters):
    #creates adjacency matrix corresponding to Small World connections
    #two cards are considered adjacent if they have exactly one type, attribute, level, atk, or def in common
    deck_monsters_array = deck_mosters.to_numpy()
    num_cards = len(deck_mosters)
    adjacency_matrix = np.zeros((num_cards,num_cards))
    for i in range(num_cards):
        card_similarities = deck_monsters_array==deck_monsters_array[i]
        similarity_measure = card_similarities.astype(int).sum(axis=1)
        adjacency_matrix[:,i] = (similarity_measure==1).astype(int) #indicates where there is exactly one similarity
    return adjacency_matrix

def find_best_bridges(deck_monster_names, required_target_names):
    #inputs: list of monster names and list of monsters that are required to connect with the small world bridges
    #output: The bridges that connect the most cards in your deck and connect with all the required targets
    deck_monster_names = list(set(deck_monster_names) | set(required_target_names)) #union names
    deck_indices = sub_df(MAIN_MONSTERS, deck_monster_names, 'name').index #indices of monsters in deck
    required_indices = sub_df(MAIN_MONSTERS, required_target_names, 'name').index #indices of required connections

    num_required_bridges = len(required_target_names) #number of cards required to connect with one bridge

    required_monster_matrix = SW_ADJACENCY_MATRIX[required_indices,:] #array corresponding to (required connection monsters) by (all monsters)
    num_bridges_to_required_cards = required_monster_matrix.sum(axis=0) #number of required connections satisfied by all monsters
    required_bridge_mask = num_bridges_to_required_cards==num_required_bridges
    bridges = MAIN_MONSTERS[required_bridge_mask] #data frame of monsters connecting all required targets
    required_bridge_indices = bridges.index #indices of monsters that satisfy all required connections
    if len(bridges)==0:
        print('There are no monsters that bridge all required targets.')
        return

    #subset of adjacency matrix corresponding to (deck monsters) by (monsters with connections to the required cards)
    bridge_matrix = SW_ADJACENCY_MATRIX[deck_indices,:][:,required_bridge_indices]

    num_deck_bridges = bridge_matrix.sum(axis=0)
    largest_num_bridges = max(num_deck_bridges)
    most_deck_bridges_mask = num_deck_bridges==largest_num_bridges
    most_deck_bridges = bridges[most_deck_bridges_mask]
    
    return most_deck_bridges

# Load Cards and Create Adjacency Matrix

In [19]:
MAIN_MONSTERS = load_main_monsters()
SW_ADJACENCY_MATRIX = create_sw_adjacency_matrix(MAIN_MONSTERS) #small world adjacency matrix of all cards

# Find Bridges With Most Connections in Deck

In [20]:
#cards that we want to find bridges between
deck_monster_names = ['Ash Blossom & Joyous Spring',
                        'Effect Veiler',
                        'Ghost Belle & Haunted Mansion',
                        'Mathmech Addition',
                        'Mathmech Circular',
                        'Mathmech Diameter',
                        'Mathmech Multiplication',
                        'Mathmech Nabla',
                        'Mathmech Sigma',
                        'Mathmech Subtraction',
                        'Parallel eXceed',
                        'Maxx "C"']

#cards that are required to connect with a bridge
required_target_names = ['Mathmech Circular',
                        'Maxx "C"',
                        'Parallel eXceed']

find_best_bridges(deck_monster_names, required_target_names)

,name,type,attribute,level,atk,def
675,Boot-Up Corporal - Command Dynamo,Machine,EARTH,4.0,0.0,2000.0
676,Boot-Up Soldier - Dread Dynamo,Machine,EARTH,4.0,0.0,2000.0
1520,Doshin @Ignister,Cyberse,EARTH,1.0,100.0,800.0
1983,Flowerdino,Dinosaur,EARTH,4.0,2000.0,0.0
2164,Gem-Turtle,Rock,EARTH,4.0,0.0,2000.0
2369,Gogogo Giant,Rock,EARTH,4.0,2000.0,0.0
2637,Heroic Challenger - Swordshield,Warrior,EARTH,4.0,0.0,2000.0
3009,Knightmare Corruptor Iblee,Cyberse,DARK,2.0,0.0,0.0
3329,Madolche Marmalmaide,Spellcaster,EARTH,4.0,800.0,2000.0
3662,Miracle Jurassic Egg,Dinosaur,EARTH,4.0,0.0,2000.0
